# Imports

In [13]:
import pandas as pd
import sqlite3
from pyspark.sql import SparkSession
import matplotlib.pyplot as plt
from io import StringIO
import requests
import os

# Initialize Spark

In [2]:
spark = SparkSession.builder.appName("AgenticAISetup").getOrCreate()
print("Spark session created ✅")

Spark session created ✅


# Load Multiple CSVs from GitHub Folder

### GitHub repo info

In [8]:
GITHUB_USER = "aniket-malpure"
REPO_NAME = "E-Commerce-Big-Data-Engineering"
BRANCH = "main"
DATA_PATH = "Data"

### List of files available in the GitHub folder

In [4]:
csv_files = [
    "olist_customers_dataset.csv",
    "olist_geolocation_dataset.csv",
    "olist_order_items_dataset.csv",
    "olist_order_payments_dataset.csv",
    "olist_order_reviews_dataset.csv",
    "olist_orders_dataset.csv",
    "olist_products_dataset.csv",
    "olist_sellers_dataset.csv"
]

### Function to load CSV from GitHub using requests

In [6]:
def load_csv_from_github(user, repo, branch, data_path, filename):
    url = f"https://raw.githubusercontent.com/{user}/{repo}/{branch}/{data_path}/{filename}"
    response = requests.get(url)
    if response.status_code == 200:
        df = pd.read_csv(StringIO(response.text))
        print(f"✅ Loaded {filename} ({len(df)} rows, {len(df.columns)} columns)")
        return df
    else:
        print(f"❌ Failed to load {filename} — HTTP {response.status_code}")
        return None

### Load all CSVs

In [11]:
dataframes = {}
for file in csv_files:
    df = load_csv_from_github(GITHUB_USER, REPO_NAME, BRANCH, DATA_PATH, file)
    if df is not None:
        dataframes[file.replace(".csv", "")] = spark.createDataFrame(df)

✅ Loaded olist_customers_dataset.csv (99441 rows, 5 columns)
✅ Loaded olist_geolocation_dataset.csv (1000163 rows, 5 columns)
✅ Loaded olist_order_items_dataset.csv (112650 rows, 7 columns)
✅ Loaded olist_order_payments_dataset.csv (103886 rows, 5 columns)
✅ Loaded olist_order_reviews_dataset.csv (99224 rows, 7 columns)
✅ Loaded olist_orders_dataset.csv (99441 rows, 8 columns)
✅ Loaded olist_products_dataset.csv (32951 rows, 9 columns)
✅ Loaded olist_sellers_dataset.csv (3095 rows, 4 columns)


# Verify one dataset (orders)

In [12]:
if "olist_orders_dataset" in dataframes:
    print("\nPreview of 'olist_orders_dataset':")
    dataframes["olist_orders_dataset"].show(5)


Preview of 'olist_orders_dataset':
+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|            order_id|         customer_id|order_status|order_purchase_timestamp|  order_approved_at|order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|
+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|e481f51cbdc54678b...|9ef432eb625129730...|   delivered|     2017-10-02 10:56:33|2017-10-02 11:07:15|         2017-10-04 19:55:00|          2017-10-10 21:25:13|          2017-10-18 00:00:00|
|53cdb2fc8bc7dce0b...|b0830fb4747a6c6d2...|   delivered|     2018-07-24 20:41:37|2018-07-26 03:24:27|         2018-07-26 14:31:00|          2018-08-07 15:27:45|          2018-08-13 00:00:00|
|47770eb9

# Connect to existing SQLite3 database

In [14]:
db_path = "olist_database.db"
if os.path.exists(db_path):
    conn = sqlite3.connect(db_path)
    print(f"\n✅ Connected to SQLite database at: {db_path}")


✅ Connected to SQLite database at: olist_database.db


In [15]:
tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)
print("Available tables in DB:")
display(tables)

Available tables in DB:


,name
0,olist_orders
